# Formular

We have: 
$
\\  \ \\
cost(h_\theta (x), y) = \left \{ \begin{array}{rcl} -log(h_\theta(x)) \end{array} \right \}
\ \\ \ \\
J = \frac{1}{2m} \sum_{i = 1}^{m} cost (h(x^{(i)}), y^{(i)} ) 
\ \\ \ \\
\rightarrow \frac{dJ}{dh} = \frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)} ) \\ 
\ \\
h(a) = \frac {1} {1 + e^{-a}} \rightarrow 
\ \\ \ \\
a_{\theta}(x) = \theta^T x \rightarrow \frac{da}{d\theta} = x\\
\ \\
\frac{dJ}{d\theta} = \frac{dJ}{dh} \frac{dh}{d\theta} = \frac{1}{m} \sum_{i=1}^{m} (h(x^{(i)}) - y^{(i)} ) x^{(i)} \\
\ \\
\theta_i = \theta_i - \alpha \frac{\partial J (\theta_0,\dots, \theta_n)} {\partial \theta_i}  
$

In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import pandas as pd

In [2]:
ex4_data = pd.read_csv('ex4data.txt', header= None).to_numpy()

In [3]:
x1, x2, y = ex4_data[:, 0], ex4_data[:, 1], ex4_data[:, 2]  

In [4]:
x = ex4_data[:, 0:2]

In [5]:
colors = np.where(y == 1, 'red', 'blue')

In [6]:
class Logistic_Regression_Multivariables:
    def __init__(self, *, number_of_feature: int) -> None:
        self.number_of_features = number_of_feature

    def normalize_vector(self, vector: np.ndarray) -> np.ndarray:
        mean = np.mean(vector)
        std = np.std(vector)
        if std == 0:
            return vector - mean
        return (vector - mean) / std
    
    def normalize_input(self, *, X: np.ndarray) -> tuple:
        norm_X = np.apply_along_axis(self.normalize_vector, arr=X, axis=0).reshape(-1, self.number_of_features)
        return norm_X

    def add_ones_columns(self, *, normalized_input: np.ndarray) -> np.ndarray:
        ones = np.ones(len(normalized_input)).reshape(-1, 1)
        x_add = np.hstack((ones, normalized_input))
        return x_add

    def predict(self, *, theta: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        y_pred = np.matmul(normalized_input, theta)
        y_pred = 1/(1 + np.exp(-y_pred))
        return y_pred
    
    def compute_loss(self, *, y_true: np.ndarray, y_pred: np.ndarray) -> np.ndarray:
        m = len(y_true)
        epsilon = 1e-15
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        J = np.sum(- y_true*np.log(y_pred) - (1 - y_true)*np.log(1 - y_pred)) / (m)
        return J
    
    def update_params(self, *, theta: np.ndarray, lr: float, y_pred: np.ndarray, 
                      y_true: np.ndarray, normalized_input: np.ndarray) -> np.ndarray:
        m = len(y_true)
        E = y_pred - y_true
        dJ_dtheta = np.dot(normalized_input.T, E) / (m)
        theta_updated = theta - lr*dJ_dtheta
        return theta_updated
    
    def train(self, *, epochs: int, theta: np.ndarray, input: np.ndarray, 
              output: np.ndarray, lr: float, plot_graph: False, color: list, time_delay: float) -> np.ndarray:
        output = output.reshape(-1, 1)
        normalized_input = self.normalize_input(X= input)
        normalized_input_with_ones = self.add_ones_columns(normalized_input= normalized_input)

        J_array = np.array([])
        
        for epoch in range(epochs):
            y_pred = self.predict(theta= theta, normalized_input= normalized_input_with_ones)

            J = self.compute_loss(y_true= output, y_pred= y_pred)
            J_array = np.append(arr= J_array, values= J)

            theta = self.update_params(theta= theta, lr= lr, y_pred= y_pred, 
                                       y_true= output, normalized_input= normalized_input_with_ones)

            
    
        return J_array, theta

In [7]:
np.random.seed(1)

theta_init = np.random.rand(3, 1)
theta_init

array([[4.17022005e-01],
       [7.20324493e-01],
       [1.14374817e-04]])

In [8]:
Logistic_Regression = Logistic_Regression_Multivariables(number_of_feature= 2)

In [9]:
epochs = 1000
learning_rate =  0.01

J_array, theta_ = Logistic_Regression.train(
    epochs=epochs, 
    theta=theta_init, 
    input=x, 
    output=y.reshape(-1, 1), 
    lr=learning_rate,
    plot_graph= False, 
    color= colors, 
    time_delay= 0.001,
)

In [10]:
theta_.shape

(3, 1)

In [11]:
normalized_input = Logistic_Regression.normalize_input(X=x)

# Generate values for Feature 1
x_values = np.linspace(normalized_input[:, 0].min(), normalized_input[:, 0].max(), 100)
y_boundary = -(theta_[0] + theta_[1] * x_values) / theta_[2]


In [12]:
fig = go.Figure()

# Plot data points
fig.add_trace(go.Scatter(
    x=normalized_input[:, 0], 
    y=normalized_input[:, 1], 
    mode='markers', 
    marker=dict(color=colors, size=10),
    name='Data Points'
))

# Plot decision boundary
fig.add_trace(go.Scatter(
    x=x_values, 
    y=y_boundary.flatten(), 
    mode='lines', 
    name='Decision Boundary', 
    line=dict(color='green', width=2)
))

In [16]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(x= np.arange(len(J_array)), y= J_array)
)